In [1]:
from functions.metabase import (
    make_metabase_session, 
    get_metabase_cards, 
    parse_metabase_cards, 
    transform_metabase_cards,
    get_cards_to_archive, 
    archive_batch,
    store_archived_cards_into_snowflake
)
import os
from dotenv import load_dotenv

load_dotenv()

METABASE_URL = "https://sfr3.metabaseapp.com"
METABASE_API_KEY = os.environ.get('METABASE_API_KEY')


In [2]:
session = make_metabase_session(METABASE_API_KEY)

In [3]:
cards = get_metabase_cards(METABASE_URL, session)

In [4]:
df = parse_metabase_cards(cards)
df.head()

,card_id,collection_id,card_name,card_sql,db_id,created_at,created_by,last_updated_at,last_updated_by,last_used_at,archived
0,571,92.0,$0 line item explorer,with projects as ( \n SELECT distinct co.pr...,35,2023-04-25T21:40:31.977831Z,Andrew Faircloth,2023-06-09T02:28:15.248034Z,anthony.marinelli@sfr3.com,2024-08-29T21:04:33.448571Z,False
1,471,128.0,0 rent leases in buildium,select * \n\nfrom {{#431-basic-lease-info}}\n\...,35,2023-04-18T16:44:15.272299Z,Andrew Nelson,2023-04-18T18:54:32.4301Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,False
2,10137,3070.0,$0 turn moveout reasons,"SELECT y.propid, y.project_id, y.metro, y.most...",35,2024-08-27T10:18:01.068658Z,Amartya Sengupta,2024-08-27T18:12:47.087135Z,amartya@sfr3.com,2024-09-23T16:05:06.915891Z,False
3,8362,NaN,100 Spokane Drive,None,35,2024-04-15T13:05:43.791332Z,Timothy Cha,2024-09-01T04:25:32.039323Z,timothy.cha@sfr3.com,2024-09-01T04:25:23.354215Z,False
4,6602,144.0,10/23 Missing Turn Speed Bonus,\nWITH bonus_scores AS (\n SELECT *\n FR...,35,2023-12-10T22:02:00.181909Z,Amartya Sengupta,2023-12-10T22:55:44.220353Z,amartya@sfr3.com,2024-08-29T21:04:33.448571Z,False


In [5]:
df_transformed = transform_metabase_cards(df)
df_transformed.head()

,card_id,collection_id,card_name,card_sql,db_id,created_at,created_by,last_updated_at,last_updated_by,last_used_at,...,raw_stitch,raw_fivetran,raw_airbyte,raw_portable,raw_dataddo,flagged,days_since_last_usage,days_since_last_update,update_category,usage_category
0,571,92.0,$0 line item explorer,with projects as ( \n SELECT distinct co.pr...,35,2023-04-25T21:40:31.977831Z,Andrew Faircloth,2023-06-09T02:28:15.248034Z,anthony.marinelli@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,563,since 500-599 days,since 110-119 days
1,471,128.0,0 rent leases in buildium,select * \n\nfrom {{#431-basic-lease-info}}\n\...,35,2023-04-18T16:44:15.272299Z,Andrew Nelson,2023-04-18T18:54:32.4301Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,614,since 600-699 days,since 110-119 days
2,10137,3070.0,$0 turn moveout reasons,"SELECT y.propid, y.project_id, y.metro, y.most...",35,2024-08-27T10:18:01.068658Z,Amartya Sengupta,2024-08-27T18:12:47.087135Z,amartya@sfr3.com,2024-09-23T16:05:06.915891Z,...,NaN,NaN,NaN,NaN,NaN,False,90,117,since 100-199 days,since 80-89 days
3,8362,NaN,100 Spokane Drive,None,35,2024-04-15T13:05:43.791332Z,Timothy Cha,2024-09-01T04:25:32.039323Z,timothy.cha@sfr3.com,2024-09-01T04:25:23.354215Z,...,NaN,NaN,NaN,NaN,NaN,False,113,113,since 100-199 days,since 110-119 days
4,6602,144.0,10/23 Missing Turn Speed Bonus,\nWITH bonus_scores AS (\n SELECT *\n FR...,35,2023-12-10T22:02:00.181909Z,Amartya Sengupta,2023-12-10T22:55:44.220353Z,amartya@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,378,since 300-399 days,since 110-119 days


In [6]:
df_transformed.shape

(7454, 24)

In [7]:
df_to_archive = get_cards_to_archive(df_transformed)
df_to_archive.head()

,card_id,collection_id,card_name,card_sql,db_id,created_at,created_by,last_updated_at,last_updated_by,last_used_at,...,raw_stitch,raw_fivetran,raw_airbyte,raw_portable,raw_dataddo,flagged,days_since_last_usage,days_since_last_update,update_category,usage_category
0,571,92.0,$0 line item explorer,with projects as ( \n SELECT distinct co.pr...,35,2023-04-25T21:40:31.977831Z,Andrew Faircloth,2023-06-09T02:28:15.248034Z,anthony.marinelli@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,563,since 500-599 days,since 110-119 days
1,471,128.0,0 rent leases in buildium,select * \n\nfrom {{#431-basic-lease-info}}\n\...,35,2023-04-18T16:44:15.272299Z,Andrew Nelson,2023-04-18T18:54:32.4301Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,614,since 600-699 days,since 110-119 days
3,8362,NaN,100 Spokane Drive,None,35,2024-04-15T13:05:43.791332Z,Timothy Cha,2024-09-01T04:25:32.039323Z,timothy.cha@sfr3.com,2024-09-01T04:25:23.354215Z,...,NaN,NaN,NaN,NaN,NaN,False,113,113,since 100-199 days,since 110-119 days
4,6602,144.0,10/23 Missing Turn Speed Bonus,\nWITH bonus_scores AS (\n SELECT *\n FR...,35,2023-12-10T22:02:00.181909Z,Amartya Sengupta,2023-12-10T22:55:44.220353Z,amartya@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,378,since 300-399 days,since 110-119 days
5,6110,1455.0,10/23 - Scorecard EOM Date Change Sweep (from ...,WITH volume_and_budget_events AS (\n SELECT...,35,2023-10-27T16:38:06.458025Z,Amartya Sengupta,2024-02-02T16:21:06.212378Z,amartya@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,115,324,since 300-399 days,since 110-119 days


In [11]:
final_output = archive_batch(df_to_archive, session, METABASE_URL)
final_msg = final_output[0]
cards_not_archived = final_output[1]
#cards_not_archived = [] for testing

In [ ]:
import datetime as dt
condition_to_log = ~df_to_archive['card_id'].isin(cards_not_archived)
df_to_log = df_to_archive[condition_to_log]
etl_timestamp = dt.datetime.now()

df_to_log['etl_timestamp'] = etl_timestamp

store_archived_cards_into_snowflake(df_to_log)